# Importing The Depedencies

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
warnings.filterwarnings("ignore")

In [3]:
movies=pd.read_csv("movies.csv")
movies

movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  
0     Adventure|Animation|Children|Comedy|Fantasy  
1                      Adventure|Children|Fantasy  
2                                  Comedy|Romance  
3                            Comedy|Drama|Romance  
4                                          Comedy  
...                                           ...  
9737              Action|Animation|Comedy|Fantasy  
9738                     Animation|Comedy|Fantasy  
9739                                        Drama  
9740                             Action|Animation  
9741                                       Comedy  

[9742 rows x 3 columns]

In [4]:
movies.shape

(9742, 3)

In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [6]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [7]:
movies.duplicated().sum()

0

In [17]:
import re
def clean_title(title):
    return re.sub("[^a-zA-Z0-9\s]","",title)

In [19]:
movies["cleaned_title"]=movies.title.apply(clean_title)

In [21]:
movies.drop("title",axis=1,inplace=True)

In [23]:
movies

movieId                                       genres  \
0           1  Adventure|Animation|Children|Comedy|Fantasy   
1           2                   Adventure|Children|Fantasy   
2           3                               Comedy|Romance   
3           4                         Comedy|Drama|Romance   
4           5                                       Comedy   
...       ...                                          ...   
9737   193581              Action|Animation|Comedy|Fantasy   
9738   193583                     Animation|Comedy|Fantasy   
9739   193585                                        Drama   
9740   193587                             Action|Animation   
9741   193609                                       Comedy   

                               cleaned_title  
0                             Toy Story 1995  
1                               Jumanji 1995  
2                      Grumpier Old Men 1995  
3                     Waiting to Exhale 1995  
4           Father of the Bride Part II 1995  
...                                      ...  
9737  Black Butler Book of the Atlantic 2017  
9738               No Game No Life Zero 2017  
9739                              Flint 2017  
9740        Bungo Stray Dogs Dead Apple 2018  
9741        Andrew Dice Clay Dice Rules 1991  

[9742 rows x 3 columns]

In [25]:
#Creating a TF-IDF matrix
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(ngram_range=(1,2))
tfidf=vectorizer.fit_transform(movies["cleaned_title"])

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
def search_title():
    title = input("Enter the movie title: ")
    title = clean_title(title)   
    vec = vectorizer.transform([title])  
    similarity = cosine_similarity(vec, tfidf).flatten()
    indices = np.argpartition(similarity, -10)[-10:]
    indices = indices[np.argsort(similarity[indices])[::-1]]
    results = movies.iloc[indices][["cleaned_title"]].copy()
    

    return results

In [189]:
search_title()

Enter the movie title:  Avatar


cleaned_title
7212                                      Avatar 2009
9741                 Andrew Dice Clay Dice Rules 1991
3251                         Beach Blanket Bingo 1965
3250                               Another Woman 1988
3249                                       Alice 1990
3248                            Rape Me Baisemoi 2000
3247                          Lost and Delirious 2001
3246                               Scary Movie 2 2001
3244                                  Cats  Dogs 2001
3252  Big Deal on Madonna Street I Soliti Ignoti 1958

# Collaborative Filtering

In [97]:
ratings=pd.read_csv("ratings.csv")
ratings

userId  movieId  rating   timestamp
0            1        1     4.0   964982703
1            1        3     4.0   964981247
2            1        6     4.0   964982224
3            1       47     5.0   964983815
4            1       50     5.0   964982931
...        ...      ...     ...         ...
100831     610   166534     4.0  1493848402
100832     610   168248     5.0  1493850091
100833     610   168250     5.0  1494273047
100834     610   168252     5.0  1493846352
100835     610   170875     3.0  1493846415

[100836 rows x 4 columns]

In [99]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [101]:
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [103]:
ratings.duplicated().sum()

0

In [105]:
movieId=5

In [107]:
similar_users=ratings[(ratings["movieId"] == movieId) & (ratings["rating"]>=4)]["userId"].unique()

In [109]:
similar_users

array([  6,  43,  58,  66, 103, 107, 120, 147, 169, 200, 337, 389],
      dtype=int64)

In [111]:
similar_users_rec=ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"]>4)]["movieId"]

In [113]:
similar_users_rec

561         3
563         5
575        22
583        36
598        86
         ... 
60313     736
60316     780
60321     805
60322     832
60323    1356
Name: movieId, Length: 691, dtype: int64

In [115]:
similar_users_rec=similar_users_rec.value_counts()/len(similar_users)
similar_users_rec=similar_users_rec[similar_users_rec>.10]
similar_users_rec.name="similar"

In [117]:
all_users=ratings[(ratings["movieId"].isin(similar_users_rec.index)) & (ratings["rating"]>4)]
all_users

userId  movieId  rating   timestamp
3            1       47     5.0   964983815
13           1      231     5.0   964981179
15           1      260     5.0   964981680
18           1      333     5.0   964981179
21           1      362     5.0   964982588
...        ...      ...     ...         ...
99993      610     6874     4.5  1493844709
100009     610     7153     5.0  1479545863
100020     610     7361     5.0  1479543278
100026     610     7438     4.5  1493844840
100310     610    58559     4.5  1493844688

[4496 rows x 4 columns]

In [141]:
all_users_recs=all_users["movieId"].value_counts()/len(all_users["userId"].unique())
all_users_recs

movieId
318     0.366606
296     0.303085
356     0.281307
2571    0.272232
2959    0.235935
          ...   
7       0.007260
262     0.007260
5       0.007260
783     0.005445
2724    0.005445
Name: count, Length: 119, dtype: float64

In [147]:
all_users_recs = all_users_recs[all_users_recs > 0.10]
all_users_recs.name = "all"

In [153]:
percentage_rec = pd.concat([similar_users_rec, all_users_recs], axis=1)

percentage_rec = percentage_rec.fillna(0)

percentage_rec["score"] = percentage_rec.apply(
    lambda row: row["similar"] / row["all"] if row["all"] > 0 else 0, axis=1
)

percentage_rec = percentage_rec.sort_values("score", ascending=False)

percentage_rec


similar       all     score
movieId                              
110      0.583333  0.183303  3.182343
150      0.250000  0.103448  2.416667
457      0.250000  0.105263  2.375000
364      0.250000  0.105263  2.375000
1704     0.250000  0.110708  2.258197
...           ...       ...       ...
168      0.250000  0.000000  0.000000
104      0.250000  0.000000  0.000000
500      0.250000  0.000000  0.000000
62       0.250000  0.000000  0.000000
552      0.166667  0.000000  0.000000

[119 rows x 3 columns]

In [155]:
percentage_rec.head(10).merge(movies ,left_index=True,right_on="movieId")

similar       all     score  movieId  \
97    0.583333  0.183303  3.182343      110   
123   0.250000  0.103448  2.416667      150   
398   0.250000  0.105263  2.375000      457   
322   0.250000  0.105263  2.375000      364   
1284  0.250000  0.110708  2.258197     1704   
4909  0.250000  0.112523  2.221774     7361   
0     0.250000  0.117967  2.119231        1   
277   0.666667  0.366606  1.818482      318   
4137  0.250000  0.145191  1.721875     5952   
43    0.250000  0.147005  1.700617       47   

                                               genres  \
97                                   Action|Drama|War   
123                              Adventure|Drama|IMAX   
398                                          Thriller   
322   Adventure|Animation|Children|Drama|Musical|IMAX   
1284                                    Drama|Romance   
4909                             Drama|Romance|Sci-Fi   
0         Adventure|Animation|Children|Comedy|Fantasy   
277                                       Crime|Drama   
4137                                Adventure|Fantasy   
43                                   Mystery|Thriller   

                                   cleaned_title  
97                               Braveheart 1995  
123                               Apollo 13 1995  
398                            Fugitive The 1993  
322                           Lion King The 1994  
1284                      Good Will Hunting 1997  
4909  Eternal Sunshine of the Spotless Mind 2004  
0                                 Toy Story 1995  
277                Shawshank Redemption The 1994  
4137   Lord of the Rings The Two Towers The 2002  
43                          Seven aka Se7en 1995

In [193]:
def find_similar_movie_by_input():
    # Ask the user to type a movie title
    title = input("🎬 Enter a movie title: ")

    # 1. Get movieId from title
    movieId = movies.loc[movies["cleaned_title"].str.contains(title, case=False, na=False), "movieId"].values
    
    if len(movieId) == 0:
        print("❌ Movie not found!")
        return None
    
    movieId = movieId[0]  # pick first match if multiple

    # 2. Find similar users who liked the same movie (rating >= 4)
    similar_users = ratings[(ratings["movieId"] == movieId) & (ratings["rating"] >= 4)]["userId"].unique()

    # 3. Movies those similar users liked (rating > 4)
    similar_users_rec = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

    # 4. Count how often each movie appears and normalize
    if len(similar_users) > 0:
        similar_users_rec = similar_users_rec.value_counts() / len(similar_users)
        similar_users_rec = similar_users_rec[similar_users_rec > 0.10]
        similar_users_rec.name = "similar"
    else:
        similar_users_rec = pd.Series(dtype="float64", name="similar")

    # 5. Get all users who rated these movies > 4
    all_users = ratings[(ratings["movieId"].isin(similar_users_rec.index)) & (ratings["rating"] > 4)]

    # 6. Popularity among all users
    if not all_users.empty:
        all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
        all_users_recs.name = "all"
    else:
        all_users_recs = pd.Series(dtype="float64", name="all")

    # 7. Combine similar vs all (handle missing values)
    percentage_rec = pd.concat([similar_users_rec, all_users_recs], axis=1).fillna(0)

    # 8. Score = friends’ preference / everyone’s preference (avoid div by zero)
    percentage_rec["score"] = percentage_rec.apply(
        lambda row: row["similar"] / row["all"] if row["all"] > 0 else 0, axis=1
    )

    # 9. Sort by best score
    percentage_rec = percentage_rec.sort_values("score", ascending=False)

    # 10. Merge with movies dataset for names (exclude the original movie itself)
    recommendations = percentage_rec.head(10).merge(movies, left_index=True, right_on="movieId", how="left")
    recommendations = recommendations[recommendations["movieId"] != movieId]

    # ✅ Print results
    print("\n🎬 Top 10 Recommended Movies:\n")
    return recommendations


In [197]:
find_similar_movie_by_input()

🎬 Enter a movie title:  Iron Man



🎬 Top 10 Recommended Movies:



similar       all      score  movieId  \
8151  0.127273  0.012216  10.418182   102125   
8425  0.163636  0.017452   9.376364   111362   
7620  0.163636  0.017452   9.376364    87232   
8693  0.109091  0.012216   8.929870   122912   
8395  0.145455  0.017452   8.334545   110102   
8159  0.109091  0.013962   7.813636   102445   
8695  0.109091  0.013962   7.813636   122918   
6170  0.145455  0.019197   7.576860    44665   
8694  0.145455  0.019197   7.576860   122916   

                                    genres  \
8151           Action|Sci-Fi|Thriller|IMAX   
8425               Action|Adventure|Sci-Fi   
7620  Action|Adventure|Sci-Fi|Thriller|War   
8693               Action|Adventure|Sci-Fi   
8395          Action|Adventure|Sci-Fi|IMAX   
8159          Action|Adventure|Sci-Fi|IMAX   
8695               Action|Adventure|Sci-Fi   
6170                   Crime|Drama|Mystery   
8694               Action|Adventure|Sci-Fi   

                                cleaned_title  
8151                          Iron Man 3 2013  
8425            XMen Days of Future Past 2014  
7620                    XMen First Class 2011  
8693       Avengers Infinity War  Part I 2018  
8395  Captain America The Winter Soldier 2014  
8159             Star Trek Into Darkness 2013  
8695           Guardians of the Galaxy 2 2017  
6170                 Lucky Number Slevin 2006  
8694                       Thor Ragnarok 2017